In [13]:
import tensorflow as tf
import numpy as np
import scipy.stats as st

In [ ]:
# Trained net f_theta
#f_theta = tensorflow.keras.models.load_model('./two_link_arm_pretrained_M.model')
f_theta = model
# Have an observation sequence z
loaded = np.load('./two_link_arm_pretraining_data.npz')

# print(np.array_equal(angles , loaded['x']))
# print(np.array_equal(positions , loaded['y']))
angles = loaded['x']
positions = loaded['y']

x = angles
z = positions
# Transition probability d
d = 0.5
# measurement incertainty Sigma
noise = 0.01
Sigma = np.identity(2)*noise
print(Sigma)

# Number of particles N
N = 15

# Number of timesteps
T = 10

#Mask shape
mask_shape= 512

In [3]:
def binarize(x):
    return 1. if(x>=0.5) else 0.

vbinarize = np.vectorize(binarize)
def flip(x):
    return 0. if x==1. else 1.

def bitflip(a, d):
    array = a
    index = np.random.choice(range(len(a)), size = int(d*len(a)), replace = False)
    for x in index:
        array[x] = flip(array[x])
    return array

def resample(mask_list, weights):
    N = len(mask_list)
    index = np.random.choice(range(0,N), size = N, replace = True, p = weights)
    return([mask_list[i] for i in index])

In [ ]:
def smcd(T, N, f_theta, z, d, Sigma, mask_shape):
    
    mask_estimate = np.zeros((T, mask_shape))
    #Initialise N random masks M0_i
    masks = np.zeros((T+1,N,mask_shape))
    for i in range(0,N):
        masks[0][i] = np.random.rand(mask_shape)
        masks[0][i] = np.array([binarize(x) for x in masks[0][i]])
    x=np.zeros((T+1,2))
    #Initialize x at z_0 ??
    x[0] = z[0]
    for t in range(1,T+1):
        w = np.zeros(N)
        for i in range(0,N):
              # Sample Mt_i ~ p(Mt_i|M(t-1)_i) by bit-flipping
            masks[t][i] = bitflip(masks[t-1][i], d)
            # Sample xt_i=f_theta(x(t-1), u_t|Mt_i)
            ##x[t][i] = f_theta.predict(x[t-1][i], u[t], masks[t][i])
            #print(x[t-1], i, t)
            x_t_i = f_theta.predict(np.array([x[t-1]])) #include mask and u here
            # Evaluate w_i = N(z_t|xt_i, Sigma)
            w[i] = st.multivariate_normal.pdf(z[t], mean = x_t_i[0], cov= Sigma)
        # Draw masks
        # print(w)
    w = w/sum(w)
    #print(w)
   # print(masks[t])
    masks[t] = resample(masks[t], w)  
   # print(masks[t])
        # Compute the nest mask estimate (mean)
    mask_estimate[t-1] = 1/N * sum(masks[t])
    mask_estimate[t-1] = vbinarize(mask_estimate[t-1])
    ## Is this how x[t] is supposed to be updated??
    x[t] = f_theta.predict(np.array([x[t-1]]))
    return (mask_estimate)

In [ ]:
smcd(T,N,f_theta, z, d, Sigma, mask_shape)